In [1]:
!pip  install langchain langchain-community langchainhub chromadb bs4 sentence_transformers pypdf

!pip install langchain
!pip install openai
!pip install langchain-huggingface
!pip install huggingface_hub
!pip install transformers
!pip install accelerate
!pip install  bitsandbytes
!pip install langchain-community langchain-core
!pip install --upgrade langchain

import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import BSHTMLLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import PromptTemplate
import pandas as pd
import string
from langchain.docstore.document import Document
import os
from langchain_huggingface import HuggingFacePipeline
from langchain_huggingface import HuggingFaceEndpoint
import re
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
import torch
from langchain_community.vectorstores import Chroma

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoConfig, pipeline
from huggingface_hub import login

sec_key = ""
os.environ["HUGGINGFACEHUB_API_TOKEN"]=sec_key

repo_id="mistralai/Mistral-7B-Instruct-v0.2"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.5/605.5 kB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.5 MB/s eta 0:00:

In [2]:


llm=HuggingFaceEndpoint(repo_id=repo_id,max_length=2048,temperature=0.1,token=sec_key)

embeddings = HuggingFaceEmbeddings()

                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.
                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.
<ipython-input-2-b164ece3c789>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
<ipython-input-2-b164ece3c789>:3: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = H

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [18]:
df = pd.read_csv("/content/final-data-test-v2.csv")

# df.head(1)

# first_10_rows = df.head(2)
# last_10_rows = df.tail(2)


# df = pd.concat([first_10_rows, last_10_rows])

df = df.head(4)

pdfs_names = list(df["pdf names"])

pdfs_names

text = list(df["text"])
len(text)


4

In [19]:
pdfs_names

['max_life_smart_secure_plus_plan_prospectus_147c9617eb.pdf',
 'smart_term_plan_prospectus_f719c98aba.pdf',
 'smart_term_plan_prospectus_f719c98aba.pdf',
 'max_life_smart_secure_plus_plan_prospectus_147c9617eb.pdf']

In [20]:
df.head(1)

,pdf names,text,Conditional Clauses,Cross-Dependent Clauses,Legal Terminology,Ambiguity in Expression,Interpretable
0,max_life_smart_secure_plus_plan_prospectus_147...,Product Name: Max Life Smart Secure Plus Plan ...,1,0,1,0,0


In [21]:
df.columns

Index(['pdf names', 'text', 'Conditional Clauses', 'Cross-Dependent Clauses',
       'Legal Terminology', 'Ambiguity in Expression', 'Interpretable'],
      dtype='object')

In [22]:
'pdf names', 'text', 'Conditional Clauses', 'Cross-Dependent Clauses','Legal Terminology', 'Ambiguity in Expression', 'Interpretable'

('pdf names',
 'text',
 'Conditional Clauses',
 'Cross-Dependent Clauses',
 'Legal Terminology',
 'Ambiguity in Expression',
 'Interpretable')

In [23]:
text = []
category_array = []


categories = ['Conditional Clauses', 'Cross-Dependent Clauses', 'Legal Terminology', 'Ambiguity in Expression', 'Interpretable']


for _, row in df.iterrows():
    # Text for the current row
    text_ = row['text']

    # List to store the categories where the value is 1
    temp_categories = []

    # Check for each category column and if value is 1, add the category to temp_categories
    for category in categories:
        if row[category] == 1:
            temp_categories.append(category)

    # Append the text and corresponding categories to the respective arrays
    text.append(text_)
    category_array.append(temp_categories)

# Display the result
print("Text Array:", text)
print("Category Array:", category_array)

Text Array: ["Product Name: Max Life Smart Secure Plus Plan  \n \n  Page 28 of 40 \n \nSection F - Annexures \nAnnexure 1 – Critical Illness Definitions & Exclusion \nACI Benefit - Definitions and Exclusions \nBelow table provides the list of critical illnesses covered: \n1. Cancer of specified severity Cancer 21. Alzheimer's Disease \n2. Angioplasty* 22. Motor Neurone Disease with Permanent Symptoms \n3. First Heart Attack – of Specified Severity 23. Multiple Sclerosis with Persisting Symptoms \n4. Open Heart Replacement or Repair of Heart Valves 24. Muscular Dystrophy \n5. Surgery to aorta Heart and Artery Benefit 25. Parkinson’s Disease \n6. Cardiomyopathy 26. Loss of Independent Existence \n7. Primary Pulmonary hypertension 27. Loss of Limbs \n8. Open Chest CABG 28.  Deafness \n9. Blindness 29. Loss of Speech Others \n10. End Stage Lung Failure 30. Medullary Cystic Disease \n11. End Stage Liver Failure  31. Systematic lupus Eryth with Renal Involvement \n12. Kidney Failure Requirin

In [24]:
# def generate_prompt(policy_sentence):
#     return f"""
# You are a legal expert tasked with improving the clarity and specificity of policy document sentences. When provided with an input sentence, your goal is to identify potential ambiguities, missing details, or generalities in the text. Based on your analysis, generate exactly 3–4 of the most relevant and helpful clarification questions to remove ambiguity and ensure the sentence is clear and precise.

# Process:
# 1. Carefully read and understand the input sentence.
# 2. Identify the most important aspects that may lead to confusion, vagueness, or misinterpretation.
# 3. Formulate a maximum of 3–4 targeted clarification questions that will help resolve these issues and make the sentence clearer.

# ### Input: {policy_sentence}

# ### Output Format:
# - The output must strictly follow this pattern:
#     - "Clarification Question 1"
#     - "Clarification Question 2"
#     - "Clarification Question 3"
#     - "Clarification Question 4" (if applicable, but no more than 4)
# """

def extract_questions(text, k = 4):
    # Use regex to match the question text, considering both formats with or without dashes
    questions = re.findall(r'[-]?\s*(.*?)(?=\?)\?', text)

    # Ensure all questions end with a question mark and return a clean list
    return [question.strip() + '?' for question in questions][:k]

In [25]:
def generate_prompt(policy_sentence: str, categories: list[str]) -> str:
    category_descriptions = "\n".join(
        [f"    {i + 1}. **{category}**" for i, category in enumerate(categories)]
    )
    return f"""
You are a legal expert tasked with improving the clarity and specificity of policy document sentences. Your goal is to identify potential ambiguities, missing details, or generalities in the sentence, particularly focusing on specific areas of misinterpretation as outlined in the given categories.



When provided with an input sentence, analyze it carefully to determine which category or categories apply. Then, generate exactly 3–4 clarification questions that target these specific areas of ambiguity, ensuring they are as helpful and relevant as possible to remove misinterpretation and improve the sentence's clarity.

### Input:
Policy Sentence: {policy_sentence}
Categories: {', '.join(categories)}

### Process:
1. Read the input policy sentence and reference the given categories for types of potential misinterpretations.
2. Identify which category or categories apply to the input sentence.
3. Based on the identified categories, formulate clarification questions that address potential ambiguities, vague terms, or unclear cross-references.
4. Limit your output to a maximum of 4 highly targeted questions.

### Output Format:
- The output must strictly follow this pattern:
    - "Clarification Question 1"
    - "Clarification Question 2"
    - "Clarification Question 3"
    - "Clarification Question 4" (if applicable, but no more than 4)

Use the structure and categories to ensure your questions are targeted, relevant, and actionable.
"""

In [27]:
cqs = []
for t,c in tqdm(zip(text, category_array)):

    t2 = generate_prompt(t,c)

    ans = llm.invoke(t2)

    ans = extract_questions(ans)

    cqs.append(ans)


0it [00:00, ?it/s]
1it [00:06,  6.81s/it]
2it [00:08,  4.05s/it]
3it [00:16,  5.45s/it]
4it [00:19,  4.79s/it]


In [28]:
print(len(cqs))

4


In [31]:
Chuck_Size = 1200
Chunk_Overlap = 200
K_ = 5

base = "/content/data/"

In [32]:
def generate_prompt2(question, retrieved_context):
    return f"""
You are a legal expert. Below is a legal question followed by relevant retrieved context. To answer the question, please follow these steps:

1. Carefully read the question and identify the key legal issue(s) being asked.
2. Review the retrieved context thoroughly. Ensure you understand the legal concepts, principles, and facts presented in the context.
3. Based on your legal expertise, synthesize the information from the context to formulate a precise answer that directly addresses the question.
4. Ensure your answer is legally sound, clear, and accurate. If the context contains gaps or ambiguities, note them and explain how they might impact your answer.

---

Question:
{question}

Retrieved Context:
{retrieved_context}

---

Answer:
"""

ANS = []
QUESTION = []
TEXT = []
base = "/content/data/"
vectorstore_cache = {}


for q_, t_, p_ in tqdm(zip(cqs, text, pdfs_names)):

    ans_temp = []
    q_temp = []


    pdf_name = os.path.splitext(p_)[0]


    if pdf_name not in vectorstore_cache:

        path = os.path.join(base, p_)


        loader = PyPDFLoader(path)
        docs = loader.load()

        text_splitter = RecursiveCharacterTextSplitter(chunk_size=Chuck_Size, chunk_overlap=Chunk_Overlap, add_start_index=True)
        splits = text_splitter.split_documents(docs)


        vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
        vectorstore_cache[pdf_name] = vectorstore
    else:

        vectorstore = vectorstore_cache[pdf_name]


    retriever = vectorstore.as_retriever(search_kwargs={"k": K_})


    for q in tqdm(q_):


        r_doc_ = retriever.invoke(q)
        r_doc = [r.page_content for r in r_doc_]


        r_doc = [ "#Context " + str(i+1) + ": " + r_doc[i] for i in range(len(r_doc))]
        context_string = "\n".join(r_doc)


        p = generate_prompt2(q, context_string)
        ans = llm.invoke(p)


        ans_temp.append(ans)
        q_temp.append(q)


    TEXT.append(t_)
    ANS.append(ans_temp)
    QUESTION.append(q_temp)




0it [00:00, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:06<00:18,  6.28s/it]

 50%|█████     | 2/4 [00:10<00:10,  5.05s/it]

 75%|███████▌  | 3/4 [00:13<00:04,  4.17s/it]

100%|██████████| 4/4 [00:16<00:00,  4.14s/it]

1it [02:36, 156.80s/it]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:02<00:08,  2.73s/it]

 50%|█████     | 2/4 [00:04<00:04,  2.41s/it]

 75%|███████▌  | 3/4 [00:08<00:02,  2.74s/it]

100%|██████████| 4/4 [00:10<00:00,  2.52s/it]

2it [05:11, 155.45s/it]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:02<00:08,  2.86s/it]

 50%|█████     | 2/4 [00:04<00:04,  2.20s/it]

 75%|███████▌  | 3/4 [00:09<00:03,  3.31s/it]

100%|██████████| 4/4 [00:12<00:00,  3.02s/it]

3it [05:23, 89.99s/it] 

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:03<00:10,  3.58s/it]

 50%|█████     | 2/4 [00:09<00:09,  4.86s/it]

 75%|███████▌  | 3/4 [00:11<00:03,  3.69s/it]

100%|██████████| 4/4 [00:13<00:00,  3.46s/i

In [33]:

def generate_rectification_prompt(input_1, input_2, input_3):
    """
    This function generates a prompt for a large language model (LLM) to rectify a potentially misinterpreted policy paragraph
    using the clarification questions and their answers, which are provided as lists of strings.

    Args:
    - input_1 (str): The potentially misinterpreted policy paragraph.
    - input_2 (list[str]): A list of clarification questions.
    - input_3 (list[str]): A list of clarification answers.

    Returns:
    - str: The generated prompt for rectification.
    """

    # Start building the prompt
    prompt = f"""
**Task:** You are a legal expert, and you are provided with a potentially misinterpreted paragraph extracted from a policy document. Additionally, you will receive a set of clarification questions along with the answers to those questions, which are designed to help guide your understanding of the paragraph. Your task is to use these inputs to rectify the paragraph, ensuring that the final output accurately reflects the original policy intent, without misinterpretations or ambiguities.

**Inputs:**
1. **Potentially Misinterpreted Paragraph (Input 1):**
   {input_1}

2. **Clarification Questions and Answers (Input 2 & Input 3):**
"""

    # Add clarification questions and answers
    for idx, (question, answer) in enumerate(zip(input_2, input_3), start=1):
        prompt += f"""
   - **Clarification Question {idx}:**
     {question}
   - **Clarification Answer {idx}:**
     {answer}
"""

    # Add task instructions and output section
    prompt += f"""
**Task Instructions:**
- Using the clarification questions and their answers, carefully analyze the potentially misinterpreted paragraph.
- Rectify the paragraph to make sure it conveys the intended meaning with clarity, aligning it with the context provided in the clarification answers.
- Ensure that the rectified paragraph maintains accuracy, is free of ambiguity, and reflects the true policy intent.

**Output:**
A rectified version of the potentially misinterpreted paragraph, corrected based on the clarification inputs provided.
"""

    return prompt


In [34]:
R_Text = []
Text = []
for t,a,q in tqdm(zip(TEXT,ANS,QUESTION)):



    text = generate_rectification_prompt(t, q, a)

    ans = llm.invoke(text)

    R_Text.append(ans)
    Text.append(t)

def summarize_legal_paragraph(input_paragraph):
    prompt = f"""
    You are given a paragraph from a legal policy document. Your task is to summarize it while preserving the essential meaning and important details. Follow these steps:

    1. **Understand the core message**: Read the paragraph carefully and identify the main provisions, rights, obligations, and conditions.
    2. **Highlight key legal terms**: Focus on any specific legal terms, dates, or actions that are crucial to the policy.
    3. **Eliminate unnecessary details**: Remove any extraneous information that doesn’t affect the main points or legal implications.
    4. **Keep it clear and concise**: Summarize the paragraph in simple, straightforward language while retaining the original meaning and intent.
    5. **Ensure accuracy**: Double-check that all essential details are included and that the summary accurately reflects the original content.

    The paragraph to summarize is as follows:

    "{input_paragraph}"

    Provide the summary below:
    """
    return prompt

S_Text = []
for t in tqdm(R_Text):

    text = summarize_legal_paragraph(t)

    ans = llm.invoke(text)
    S_Text.append(ans)


0it [00:00, ?it/s]
1it [00:07,  7.56s/it]
2it [00:12,  5.88s/it]
3it [00:19,  6.37s/it]
4it [00:27,  6.76s/it]

100%|██████████| 4/4 [00:12<00:00,  3.10s/it]


In [35]:
for i in range(len(pdfs_names)):

    print(len(Text[i]),len(R_Text[i]), len(S_Text[i]))

1160 2004 1605
625 1332 625
1090 1811 944
1187 2314 670


In [38]:
i = 0

In [39]:
Text[i]

"Product Name: Max Life Smart Secure Plus Plan  \n \n  Page 28 of 40 \n \nSection F - Annexures \nAnnexure 1 – Critical Illness Definitions & Exclusion \nACI Benefit - Definitions and Exclusions \nBelow table provides the list of critical illnesses covered: \n1. Cancer of specified severity Cancer 21. Alzheimer's Disease \n2. Angioplasty* 22. Motor Neurone Disease with Permanent Symptoms \n3. First Heart Attack – of Specified Severity 23. Multiple Sclerosis with Persisting Symptoms \n4. Open Heart Replacement or Repair of Heart Valves 24. Muscular Dystrophy \n5. Surgery to aorta Heart and Artery Benefit 25. Parkinson’s Disease \n6. Cardiomyopathy 26. Loss of Independent Existence \n7. Primary Pulmonary hypertension 27. Loss of Limbs \n8. Open Chest CABG 28.  Deafness \n9. Blindness 29. Loss of Speech Others \n10. End Stage Lung Failure 30. Medullary Cystic Disease \n11. End Stage Liver Failure  31. Systematic lupus Eryth with Renal Involvement \n12. Kidney Failure Requiring Regular Dia

In [40]:
R_Text[i]

'\nProduct Name: Max Life Smart Secure Plus Plan\n\nPage 28 of 40\n\nSection F - Annexures\nAnnexure 1 – Critical Illness Definitions & Exclusions\nACI Benefit - Definitions and Exclusions\n\nThe Accelerated Critical Illness (ACI) Benefit is a rider that can be attached to the base policy during the premium payment term, subject to underwriting requirements being met. The ACI Benefit can be triggered when the policy is in-force and the policyholder is diagnosed with any of the specified Critical Illnesses during the Cover period. The ACI Sum Assured can be chosen between a minimum of 5 lakhs and a maximum of 50 lakhs, which is 50% of the base policy Sum Assured chosen at inception or 50 lakhs, whichever is lower. The ACI Sum Assured is payable on the first occurrence of any of the specified Critical Illnesses, and the policy will continue with the Death Benefit Sum Assured reduced by the extent of the ACI Sum Assured paid. However, the following conditions must be met for the ACI Benef

In [41]:
S_Text[i]

'\n    The Max Life Smart Secure Plus Plan includes an optional Accelerated Critical Illness (ACI) rider, which can be added during the premium payment term, subject to underwriting requirements. The ACI rider provides coverage for specified critical illnesses and can be triggered when the policyholder is diagnosed with one of these conditions during the Cover period. The ACI Sum Assured can be chosen between 5 lakhs and 50 lakhs, which is 50% of the base policy Sum Assured or 50 lakhs, whichever is lower. The ACI Sum Assured is paid upon the first occurrence of a critical illness and reduces the Death Benefit Sum Assured. The following conditions must be met for the ACI Benefit to be triggered:\n\n    1. The ACI Benefit term cannot exceed the Premium Payment Term of the Base Cover.\n    2. ACI Benefit is not available with Single pay, 5 Pay premium payment terms, or Increasing Life Cover death benefit.\n    3. A waiting period of 90 days applies after the date of commencement of risk 

In [ ]:
DF = {
    "Text" : Text,
    "S_Text" : S_Text
}

DF = pd.DataFrame(DF)

DF.to_csv("results2.csv", index = False)

In [ ]:
You are a legal expert, you are given two paragraphs, #paragraph 1 and #paragraph 2.

go through those paragraphs, read them any mention is the two paragraphs are same or not, in terms of the context. I want the answer in either "yes" or "no"

after that, mention if a reader reads these two paragraph, which paragraph there is a high chance the reader will misinterprete

#paragraph 1: You are a legal expert, you are given two paragraphs, #paragraph 1 and #paragraph 2.

go through those paragraphs, read them any mention is the two paragraphs are same or not, in terms of the context. I want the answer in either "yes" or "no"

after that, mention if a reader reads these two paragraph, which paragraph there is a high chance the reader will misinterprete

#paragraph 1: consultant neurologist and this condition has to be medically documented for at least one (1) month with no hope of
recovery.

15. Benign Brain Tumour

A life threatening, non -cancerous tumor in the brain, cranial nerves or meninges within the skull. The presence of the
underlying tumor must be confirmed by imaging studies such as CT scan or MRI.

This brain tumor must result in at least one of the following and must be confirmed by the relevant medical specialist:

a. Permanent Neurological deficit with persisting clinical symptoms for a continuous period of at least 90 consecutive
days or
b. Undergone surgical resection or radiation therapy to treat the brain tumor.

The following conditions are excluded:
a. Cysts, Granulomas, malformations in the arteries or veins of the brain, hematomas, abscesses, pituitary tumors,
tumors of skull bones and tumors of the spinal cord.

16. Brain Surgery

The actual undergoing of surgery to the brain under general anaesthesia during which a craniotomy is performed. Keyhole
surgery is included however, minimally invasive treatment where no surgical incision is performed to expose the target,
#paragraph 2:
    A consultant neurologist must confirm a neurological condition caused by a benign brain tumor, stroke, or other specified neurological conditions. The diagnosis requires imaging studies and the condition must result in a permanent neurological deficit or treatment. Conditions like cysts, granulomas, and skull tumors are excluded. The policy defines "no hope of recovery" as the lapse of the policy for five years or the end of the term, with no opportunity to revive it.


In [ ]:
You are a legal expert, you are given a csv file, which has three columns "Text"  , "S_Text",

compare the each columns, that means

check if every corresponding Text[i] and S_Text[i] is same is terms of the context and information provided,  I want the answer in either "yes" or "no"

after that, mention if a reader reads these two paragraph (i.e Text[i] and S_Text[i]), which paragraph there is a high chance the reader will misinterprete





